这一节我们利用DGL来实现一下PageRank算法，关于PageRank的原理，大家可以参考我在ML_Notes中的这个[>>note](https://nbviewer.jupyter.org/github/zhulei227/ML_Notes/blob/master/notebooks/12_07_PGM_%E9%A9%AC%E5%B0%94%E7%A7%91%E5%A4%AB%E9%93%BE_PageRank%E7%AE%97%E6%B3%95.ipynb)  
![avatar](./pic/pagerank_demo1.png) 

如图，以节点A举例，他的PR更新公示可以表示为：   

$$
PR_A^{(t+1)}=w_{BA}PR_B^{(t)}+w_{CA}PR_C^{(t)}
$$  

这里，$w_{BA}$表示边$(B,A)$上的权重，在默认情况下，我们可以设置为节点$B$的出度的倒数，而$PR_{*}$则表示某节点的PR值，下面利用DGL来计算PR，节点编号0 ~ 3分别对应上面的节点A ~ D

In [1]:
import dgl
import torch

Using backend: pytorch


**构建图**

In [2]:
u=torch.tensor([0,0,0,1,1,2,3,3])
v=torch.tensor([1,2,3,0,3,0,1,2])
g=dgl.graph((u,v))

**计算边权重**

In [3]:
g.ndata["out_degrees"]=g.out_degrees()

In [4]:
def message_func(edges):
    return {"weight":1.0/edges.src["out_degrees"]}
g.apply_edges(message_func)
g.edata["weight"]

tensor([0.3333, 0.3333, 0.3333, 0.5000, 0.5000, 1.0000, 0.5000, 0.5000])

**计算PR**

In [5]:
#随机初始化一组PR
prs=torch.exp(torch.randn(4))
prs=prs/torch.sum(prs)
prs

tensor([0.1702, 0.3912, 0.1391, 0.2994])

In [6]:
g.ndata["pr"]=prs

In [7]:
for _ in range(100):
    g.update_all(dgl.function.u_dot_e("pr","weight","weighted_pr"),#将源节点上的pr与边上的weight相乘后放到边上的weighted_pr变量中
                 dgl.function.sum("weighted_pr","pr"))#将所有入边上的weighted_pr求和后赋值到目标节点的pr变量中

In [8]:
g.ndata["pr"]

tensor([0.3333, 0.2222, 0.2222, 0.2222])

上面的实现不够严谨，为了防止没有出度或者没有入度的节点，其实需要在聚合的时候再添加一个平滑项（见上面链接中的note），接下来继续正题，看看GNN如何用DGL来实现~